In [78]:
# !pip install sentence_transformers
# !pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118
# !pip3 install black[jupyter]


In [79]:
import pandas as pd
import os
# import nltk
import re
from transformers import DistilBertModel
from sentence_transformers import SentenceTransformer
import torch
# import torchvision
import numpy as np


In [80]:
# Modified version of Document where tokens are not included 
class Document:
  def __init__(self, doc_no, doc_text, vector):
    self.doc_no = doc_no
    self.doc_text = doc_text
    # self.tokens = tokens
    self.vector = vector

  def __str__(self):
    # return 'Document Number: ' + self.doc_no + '\nDocument Text: ' + self.doc_text + '\nTokens: ' + str(self.tokens) + '\n'
    return 'Document Number: ' + self.doc_no + '\nDocument Text: ' + self.doc_text  + '\n Vectors: ' + str(self.vector) + '\n'


  def to_dict(self):
    # return {'docno': self.doc_no, 'doctext': self.doc_text, 'tokens': self.tokens, 'text': ' '.join(self.tokens)}
    return {'docno': self.doc_no, 'doctext': self.doc_text, 'vector': self.vector.tolist()}


In [81]:
# Modified version of preprocess where tokenizing is removed 
def preprocess(file):
    with open(file, "r") as f:
        content = f.read()
    documents = re.findall(r'<DOC>(.*?)</DOC>', content, re.DOTALL)
    preprocessed_documents = []
    for document in documents:
        # Get the document number and text
        raw_no = re.search(r'<DOCNO>(.*?)</DOCNO>', document, re.DOTALL)
        doc_no = raw_no.group(1) if raw_no else ''
        raw_text = re.search(r'<TEXT>(.*?)</TEXT>', document, re.DOTALL)
        doc_text = raw_text.group(1) if raw_text else ''
        doc = Document(doc_no, doc_text, None)
        preprocessed_documents.append(doc)
    return preprocessed_documents


In [82]:
# main function to preprocess a directory of text files
def preprocess_directory(directory, num_files=-1):
  preprocessed_documents = [] 
  ctr = 0
  for filename in os.listdir(directory):
    print('Preprocessing file: ', filename)
    file = os.path.join(directory, filename)
    preprocessed_documents.extend(preprocess(file))
    ctr += 1
    if ctr == num_files and num_files != -1:
      break
  return preprocessed_documents


In [83]:

# Grabs the queries?
def extract_topics(file):
    with open(file, "r") as f:
        topic_content = f.read()
    all_topics = []
    topics = re.findall(r'<top>(.*?)</top>', topic_content, re.DOTALL)
    for topic in topics:
        raw_title = re.search(r'<title>(.*?)\n\n', topic, re.DOTALL)
        title = raw_title.group(1) if raw_title else ''
        all_topics.append(title)
    return all_topics


In [84]:
extracted_documents = []
extracted_documents = preprocess_directory('AP_collection\coll')

Preprocessing file:  AP880212
Preprocessing file:  AP880213
Preprocessing file:  AP880214
Preprocessing file:  AP880215
Preprocessing file:  AP880216
Preprocessing file:  AP880217
Preprocessing file:  AP880218
Preprocessing file:  AP880219
Preprocessing file:  AP880220
Preprocessing file:  AP880221
Preprocessing file:  AP880222
Preprocessing file:  AP880223
Preprocessing file:  AP880224
Preprocessing file:  AP880225
Preprocessing file:  AP880226
Preprocessing file:  AP880227
Preprocessing file:  AP880228
Preprocessing file:  AP880229
Preprocessing file:  AP880301
Preprocessing file:  AP880302
Preprocessing file:  AP880303
Preprocessing file:  AP880304
Preprocessing file:  AP880307
Preprocessing file:  AP880308
Preprocessing file:  AP880309
Preprocessing file:  AP880310
Preprocessing file:  AP880311
Preprocessing file:  AP880312
Preprocessing file:  AP880313
Preprocessing file:  AP880314
Preprocessing file:  AP880315
Preprocessing file:  AP880316
Preprocessing file:  AP880317
Preprocess

In [85]:
# print(extracted_documents[0].doc_no + "\n")
# print(extracted_documents[0].doc_text) 

In [86]:
topics = extract_topics("topics1-50.txt")


# DistillBERT Setup (with Cuda)


In [87]:
model = SentenceTransformer('all-distilroberta-v1', device="cuda:0")


In [88]:
torch.cuda.is_available()

True

In [89]:
torch.cuda.current_device()

0

In [90]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070 Ti Laptop GPU'

In [91]:
import datetime
import pytz

# Set the timezone to Eastern Standard Time (EST)
tz = pytz.timezone('US/Eastern')

def print_time_est():
    # Get the current time in EST
    est_time = datetime.datetime.now(tz)

    # Print the current EST time
    print("Current EST time:", est_time)


In [92]:
print_time_est()

Current EST time: 2023-04-02 14:58:56.263983-04:00


In [93]:
# for doc in extracted_documents: 
#     doc.vector = model.encode([doc.doc_text])
#     print(doc.doc_no + " is done.")

In [94]:
print_time_est()

Current EST time: 2023-04-02 14:58:56.371582-04:00


In [95]:
len(extracted_documents)

79923

In [96]:
extracted_documents[79922].vector

In [97]:
from sklearn.metrics.pairwise import cosine_similarity


In [98]:
extracted_documents[0].vector = model.encode(extracted_documents[0].doc_text)

# Output to File 


In [63]:
# LEGACY CODE 
# import json 

# with open("embedding_saves/distilroberta.json", "w") as outfile:
#     # for doc in extracted_documents:
#     #     json.dump(doc.to_dict(), outfile)
#     json.dump(extracted_documents[0].to_dict(),outfile)

### NOT compressed (too big for Github)

#### Possible csv write *(not compressed)


In [64]:
import csv

# assuming you have a list of Document objects called documents
# and assuming you have already populated the vector attribute of each Document object

# define the headers for your CSV file
headers = ['doc_no', 'doc_text', 'vector']

# open the CSV file in 'w' mode and write the headers
with open("embedding_saves/distilroberta.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)

    # loop through each Document object and write its attributes to the CSV file
    for document in extracted_documents:
        writer.writerow([document.doc_no, document.doc_text, document.vector.tolist() if document.vector is not None else None])


#### Possible csv read *(not compressed)


In [65]:
# read the CSV file and create new Document objects
extracted_documents = []
# --------------------------------------------------------------
# CHANGE THIS TO extracted_documents later 
# --------------------------------------------------------------
with open("embedding_saves/distilroberta.csv", mode='r') as file:
    reader = csv.reader(file)
    headers = next(reader) # skip the header row

    for row in reader:
        doc_no = row[0]
        doc_text = row[1]
        vector = np.array(row[2], dtype=float)
        document = Document(doc_no, doc_text, vector)
        extracted_documents.append(document)


ValueError: could not convert string to float: '[-0.010395441204309464, -0.08126188069581985, 0.0030520400032401085, -0.051090486347675323, 0.059318866580724716, 0.08064533770084381, -0.0001100039662560448, -0.008711610920727253, -0.014506327919661999, -0.004800998605787754, -0.03369065374135971, 0.0008512154454365373, -0.036835089325904846, -0.04161018505692482, -0.009351484477519989, -0.05484415218234062, 0.013274725526571274, -0.04045410826802254, 0.023280872032046318, 0.043757013976573944, -0.00033808808075264096, -0.03674377501010895, -0.012431818060576916, -0.001356494496576488, -0.03808555379509926, -0.0033934894017875195, 0.002981443190947175, 0.021589577198028564, 0.001361951231956482, 0.04464486241340637, -0.006879636086523533, 0.015462028793990612, -0.004351004026830196, 0.03017631359398365, -0.023280281573534012, 0.052706409245729446, 0.061508309096097946, -0.021434402093291283, 0.03548242524266243, 0.0006167553947307169, 0.019133636727929115, 0.07385644316673279, 0.0648852

### Compressed CSV


In [105]:
import csv
import gzip

# assuming you have a list of Document objects called documents
# and assuming you have already populated the vector attribute of each Document object

# define the headers for your CSV file
headers = ['doc_no', 'doc_text', 'vector']

# open the CSV file in 'w' mode and write the headers
with open("embedding_saves/distilroberta.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)

    # loop through each Document object and write its attributes to the CSV file
    for document in extracted_documents:
        writer.writerow([document.doc_no, document.doc_text, document.vector.tolist() if document.vector is not None else None])

# gzip the CSV file
with open("embedding_saves/distilroberta.csv", 'rb') as f_in, gzip.open("embedding_saves/distilroberta.csv.gz", 'wb') as f_out:
    f_out.writelines(f_in)



In [76]:
import csv
import gzip

# read the gzip file and create new Document objects
extracted_documents = []
with gzip.open("embedding_saves/distilroberta.csv.gz", mode='rb') as file:
    # read the uncompressed content of the gzip file
    uncompressed_content = file.read()

    # parse the uncompressed content as a CSV file
    csv_content = uncompressed_content.decode('utf-8')
    reader = csv.reader(csv_content.splitlines())

    # extract the header row
    headers = next(reader)

    # loop through each row and create a new Document object
    for row in reader:
        doc_no = row[0]
        doc_text = row[1]
        vector = np.array(row[2], dtype=float)
        document = Document(doc_no, doc_text, vector)
        extracted_documents.append(document)


In [77]:
extracted_documents[0]

IndexError: list index out of range